In [147]:
import json
import os
import googleapiclient.discovery
import re
import pandas as pd

In [155]:
DEVELOPER_KEY = ""

In [4]:
def get_comments_of_video(videoId, pageToken = None):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="snippet",
        maxResults=100,
        order="relevance",
        textFormat="plainText",
        videoId=videoId,
        pageToken=pageToken
    )
    response = request.execute()

    return response

In [50]:
def get_comments(commentId):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.comments().list(
        part="snippet",
        parentId=commentId
    )
    response = request.execute()

    return response

In [7]:
def get_all_comments(videoId):
    items = []

    page = get_comments_of_video(videoId)
    items += page['items']

    while 'nextPageToken' in page:
        page = get_comments_of_video(videoId, page['nextPageToken'])
        items += page['items']
        
    return items

In [13]:
with open('playlists_with_items.json') as f:
    playlists_with_items = json.loads(f.read())

In [17]:
suttas_an = playlists_with_items[7]
suttas_an[0]

{'kind': 'youtube#playlist',
 'etag': 'sRJbdFZ-z0vLBALaSWLhqmjuPEA',
 'id': 'PL4j9bs3T1aPQqT0FI8CNcCi2UYfLxmT6p',
 'snippet': {'publishedAt': '2017-10-09T10:20:30Z',
  'channelId': 'UCFVuMJgj2QWlg-6JZ1ye9qA',
  'title': 'Sutta Studies: Aṅguttara Nikāya',
  'description': "This collection contains videos on different suttas of the Aṅguttara Nikāya.\nMost talks are followed by a q&a session.\n\nSuttas can be found here: http://suttacentral.net\nand different English translations are provided by Obo's web (http://obo.genaud.net/backmatter/indexes/sutta/sutta_toc.htm).\n\n#buddhisttexts\n#suttastudies\n#q&a",
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/H32rY8JRB5g/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/H32rY8JRB5g/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/H32rY8JRB5g/hqdefault.jpg',
    'width': 480,
    'height': 360},
   'standard': {'url': 'https://i.ytimg.com/

In [52]:
comments_an = {}
comments_an_medhini = {} 

for video in suttas_an[1]:
    videoId = video['contentDetails']['videoId']
    comments_an[videoId] = get_all_comments(videoId)
    
    for comment in comments_an[videoId]:
        author = comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
        
        if author == 'Medhiṇī':
            comments_an_medhini[videoId] = (comment, get_comments(comment['id']))

In [53]:
with open ('comments_an.json', 'w') as f:
    f.write(json.dumps(comments_an))
    
with open ('comments_an_medhini.json', 'w') as f:
    f.write(json.dumps(comments_an_medhini))

In [110]:
mapping_medhini = {}

for videoId, (comment, replies) in comments_an_medhini.items():
    text = comment['snippet']['topLevelComment']['snippet']['textDisplay'].split('\n')[0]
    sutta_id = ''
    
    id_match = re.search('an [0-9]+\.[0-9]+( - an [0-9]+\.[0-9]+)?', text, re.I)
    if id_match:
        sutta_id = id_match[0]
      
    link = ''
    
    if replies['items']:
        link = replies['items'][0]['snippet']['textDisplay'].split('href="')[1].split('"')[0]

    mapping_medhini[videoId] = (sutta_id, text, link)

In [93]:
with open ('mapping_medhini.json', 'w') as f:
    f.write(json.dumps(mapping_medhini))

In [94]:
# infer mapping from title

In [112]:
mapping_title = {}

for video in suttas_an[1]:
    videoId = video['contentDetails']['videoId']
    title = video['snippet']['title']
    
    id_match = re.search('an [0-9]+\.[0-9]+', title, re.I)
    if id_match:
        mapping_title[videoId] = id_match[0]

In [113]:
mapping_title

{'OuAbZjcaRPk': 'AN 5.101',
 '3XCZcQz4bgA': 'AN 5.105',
 'O6-916cYaJw': 'AN 5.154',
 'O54RrN6qmPM': 'AN 5.200',
 'iQu7FAGD8D8': 'AN 5.216',
 'XWSUNuwk-0w': 'AN 6.22',
 '3m6tonrhIdA': 'AN 6.35',
 'h_blbg53BL8': 'AN 6.45',
 'TCw6aEbRwwQ': 'AN 6.53'}

In [148]:
columns = ['videoLink', 'title', 'description', 'sutta', 'publishedAt', 'link']
res = []

for video in sorted(suttas_an[1], key=lambda x: x['contentDetails']['videoPublishedAt']):
    videoId = video['contentDetails']['videoId']
    videoPublishedAt = video['contentDetails']['videoPublishedAt']
    sutta = video['contentDetails']['note'].split('| ')[1]
    title = video['snippet']['title']
    description = video['snippet']['description']
    link = '' 
    
    if videoId in mapping_medhini and mapping_medhini[videoId][2]:
        link = mapping_medhini[videoId][2]
    
    res.append({
        'video': 'https://youtu.be/' + videoId,
        'title': title,
        'description': description,
        'sutta': sutta,
        'publishedAt': videoPublishedAt,
        'link': link
    })

In [150]:
df = pd.DataFrame(res)

In [152]:
df

,video,title,description,sutta,publishedAt,link
0,https://youtu.be/XXGob52-b8c,Second Life: What is the Buddha's Teaching?,A morning session on Second Life ( http://www....,Saṃkhitta Sutta (AN 8.53),2010-09-02T12:01:46Z,
1,https://youtu.be/UZ5qDqXaChc,Come!,Come!,Quote is from the Andhakavinda Sutta (AN 5.114),2016-02-22T02:33:31Z,http://what-buddha-said.net/Canon/Sutta/KN/Dha...
2,https://youtu.be/maSyPJIhM38,Dozing,Dozing,Quote is from the Pacalāyana Sutta (AN 7.58),2016-03-15T02:43:59Z,http://obo.genaud.net/dhamma-vinaya/ati/an/07_...
3,https://youtu.be/HSWoECrsI48,The Right Time,The Right Time,Quote is from the Asamaya - Samaya Sutta (AN 5...,2016-03-19T01:25:26Z,http://obo.genaud.net/dhamma-vinaya/pts/an/05_...
4,https://youtu.be/GwCCiaNLgF8,Nothing But the Mind,Nothing But the Mind,Akammanīya Vagga (AN 1.21–30),2016-03-26T02:02:50Z,http://obo.genaud.net/dhamma-vinaya/pali/an/01...
...,...,...,...,...,...,...
74,https://youtu.be/3m6tonrhIdA,Leading To Knowledge (AN 6.35),Live stream teaching and Q&A at 9 PM Eastern T...,Vijjābhāgiya Sutta (AN 6.35),2016-10-24T02:18:17Z,
75,https://youtu.be/h_blbg53BL8,Karmic Debt (AN 6.45),Live stream teaching and Q&A at 9 PM Eastern T...,Iṇa Sutta (AN 6.45),2016-10-26T02:51:53Z,
76,https://youtu.be/TCw6aEbRwwQ,Vigilance (AN 6.53),Live stream teaching and Q&A at 9 PM Eastern T...,Appamāda sutta (AN 6.53),2016-10-28T02:27:34Z,
77,https://youtu.be/XvO7BkD0baQ,The Ocean,Join us (almost) every night at 9 PM Eastern t...,Pahārāda Sutta (AN 8.19),2016-12-14T11:57:55Z,https://suttacentral.net/an8.19/en/bodhi


In [154]:
df.to_excel('suttas_an.xlsx')